# 1.保存参数

In [19]:
import tensorflow as tf 
from tensorflow import keras

In [20]:
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.
    x = tf.reshape(x,[-1,784])
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=10)
    return x,y

In [21]:
(x_train,y_train),(x_val,y_val)=keras.datasets.fashion_mnist.load_data()
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(200).map(preprocess).shuffle(10000)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(200).map(preprocess).shuffle(10000)

db_iter = iter(db_train)
sample = next(db_iter)
print(sample[0].shape,sample[1].shape)

(200, 784) (200, 10)


In [22]:
model = keras.Sequential([
    keras.layers.Dense(512,activation=tf.nn.relu),
    keras.layers.Dense(256,activation=tf.nn.relu),
    keras.layers.Dense(128,activation=tf.nn.relu),
    keras.layers.Dense(64,activation=tf.nn.relu),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])

In [23]:
model.build(input_shape=[None,784])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             multiple                  401920    
_________________________________________________________________
dense_21 (Dense)             multiple                  131328    
_________________________________________________________________
dense_22 (Dense)             multiple                  32896     
_________________________________________________________________
dense_23 (Dense)             multiple                  8256      
_________________________________________________________________
dense_24 (Dense)             multiple                  650       
Total params: 575,050
Trainable params: 575,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.01),loss=tf.losses.categorical_crossentropy,metrics=['accuracy'])
model.fit(db_train,epochs=10,validation_data=db_val,validation_freq=2)

Epoch 1/10
300/300 [==============================] - 2s 8ms/step - loss: 1.3815 - accuracy: 0.4151
Epoch 2/10
300/300 [==============================] - 2s 8ms/step - loss: 0.7237 - accuracy: 0.7356 - val_loss: 0.6556 - val_accuracy: 0.7721
Epoch 3/10
300/300 [==============================] - 2s 7ms/step - loss: 0.6045 - accuracy: 0.7877
Epoch 4/10
300/300 [==============================] - 2s 8ms/step - loss: 0.5472 - accuracy: 0.8084 - val_loss: 0.5352 - val_accuracy: 0.8141
Epoch 5/10
300/300 [==============================] - 2s 7ms/step - loss: 0.5118 - accuracy: 0.8201
Epoch 6/10
300/300 [==============================] - 2s 8ms/step - loss: 0.4877 - accuracy: 0.8285 - val_loss: 0.4961 - val_accuracy: 0.8264
Epoch 7/10
300/300 [==============================] - 2s 7ms/step - loss: 0.4695 - accuracy: 0.8353
Epoch 8/10
300/300 [==============================] - 2s 8ms/step - loss: 0.4550 - accuracy: 0.8404 - val_loss: 0.4754 - val_accuracy: 0.8344
Epoch 9/10
300/300 [============

In [25]:
model.save_weights('./ModelData/model_weights.ckpt')

In [26]:
#模型已经删除，不能预测
del model
model.evaluate(db_val)

NameError: name 'model' is not defined

In [27]:
#重新定义模型，载入训练好的系数，进行预测
model = keras.Sequential([
    keras.layers.Dense(512,activation=tf.nn.relu),
    keras.layers.Dense(256,activation=tf.nn.relu),
    keras.layers.Dense(128,activation=tf.nn.relu),
    keras.layers.Dense(64,activation=tf.nn.relu),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.01),loss=tf.losses.categorical_crossentropy,metrics=['accuracy'])
model.load_weights('./ModelData/model_weights.ckpt')
model.evaluate(db_val)

50/50 [==============================] - 1s 11ms/step - loss: 0.4594 - accuracy: 0.8441


[0.4594354289770126, 0.838]

# 2.保存整个网络

In [28]:
model.save('./ModelData/total_model.h5')
del model
model.evaluate(db_val)

NameError: name 'model' is not defined

In [29]:
model = keras.models.load_model('./ModelData/total_model.h5')
model.evaluate(db_val)

50/50 [==============================] - 1s 10ms/step - loss: 0.4594 - accuracy: 0.8441


[0.4594354289770126, 0.838]